# Previsão de preços com vizinhança

## Neste projeto, utilizaremos a variável 'neighborhood' para prever os preços e realizaremos algumas análises exploratórias (EDA).

In [ ]:
!pip install category_encoders

### Importação de bibliotecas

In [2]:
import warnings
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from category_encoders import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted

warnings.simplefilter(action="ignore", category=FutureWarning)

### Preparando os dados

Importando os dados

In [8]:
def wrangle(filepath):
    # Read CSV file
    df = pd.read_csv(filepath)

    # Subset data: Apartments in "Capital Federal", less than 400,000
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    mask_apt = df["property_type"] == "apartment"
    mask_price = df["price_aprox_usd"] < 400_000
    df = df[mask_ba & mask_apt & mask_price]

    # Subset data: Remove outliers for "surface_covered_in_m2"
    low, high = df["surface_covered_in_m2"].quantile([0.1, 0.9])
    mask_area = df["surface_covered_in_m2"].between(low, high)
    df = df[mask_area]

    # Split "lat-lon" column
    df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
    df.drop(columns="lat-lon", inplace=True)

    # Extract neighborhood
    df["neighborhood"] = df["place_with_parent_names"].str.split("|", expand=True)[3]
    df.drop(columns="place_with_parent_names", inplace=True)

    return df

No projeto anterior, usamos nossa função Wrangle para importar dois arquivos CSV como DataFrames. Mas e se tivéssemos centenas de arquivos CSV para importar? Colocá-los em Wrangle um por um não seria uma opção. Então, vamos começar com uma técnica para ler vários arquivos CSV em um único DataFrame.

O primeiro passo é reunir os nomes de todos os arquivos que queremos importar. Podemos fazer isso usando correspondência de padrões.

In [6]:
files = glob("/content/buenos-aires-real-estate-*.csv")
files

['/content/buenos-aires-real-estate-2.csv',
 '/content/buenos-aires-real-estate-4.csv',
 '/content/buenos-aires-real-estate-1.csv',
 '/content/buenos-aires-real-estate-5.csv',
 '/content/buenos-aires-real-estate-3.csv']

O próximo passo é ler cada um dos CSVs nos arquivos para um DataFrame e colocar todos esses DataFrames em uma lista.

In [17]:
frames = [wrangle(file) for file in files]
print("Total de data frames: ", len(frames))
for frame in frames:
    print(frame.shape)

Total de data frames:  5
(1315, 17)
(1305, 17)
(1343, 17)
(1331, 17)
(1288, 17)


O passo final é usar o pandas para combinar todos os DataFrames em frames.

In [18]:
df = pd.concat(frames, ignore_index=True)
print(df.shape)
df.head()

(6582, 17)


,operation,property_type,place_with_parent_names,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon
0,sell,apartment,|Argentina|Capital Federal|Recoleta|,215000.0,USD,3259916.00,215000.00,40.0,35.0,5375.000000,6142.857143,NaN,1.0,3500.0,http://recoleta.properati.com.ar/12j4v_venta_d...,-34.588993,-58.400133
1,sell,apartment,|Argentina|Capital Federal|Recoleta|,341550.0,USD,5178717.72,341550.00,NaN,90.0,NaN,3795.000000,8.0,2.0,NaN,http://recoleta.properati.com.ar/100t0_venta_d...,-34.588044,-58.398066
2,sell,apartment,|Argentina|Capital Federal|Monserrat|,1386000.0,ARS,1382153.13,91156.62,39.0,33.0,2337.349231,42000.000000,NaN,NaN,NaN,http://monserrat.properati.com.ar/t05l_venta_d...,-34.623320,-58.397461
3,sell,apartment,|Argentina|Capital Federal|Belgrano|,105000.0,USD,1592052.00,105000.00,NaN,33.0,NaN,3181.818182,1.0,1.0,NaN,http://belgrano.properati.com.ar/zsd5_venta_de...,-34.553897,-58.451939
4,sell,apartment,|Argentina|Capital Federal|Villa del Parque|,89681.0,USD,1359779.19,89681.00,46.0,39.0,1949.586957,2299.512821,NaN,1.0,1500.0,http://villa-del-parque.properati.com.ar/12q2f...,-34.628813,-58.472230


### Explore

Observando a saída da chamada df.head() acima, vemos que precisamos fazer uma pequena limpeza antes de podermos trabalhar com as informações de vizinhança neste conjunto de dados. A boa notícia é que, como estamos usando uma função Wrangle, precisamos apenas alterá-la para limpar novamente todos os nossos arquivos CSV. É por isso que as funções são tão úteis.